## This script loads the current model and performs an evaluation of it

### Initialize
First, initialize the model with all parameters


In [3]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import sys
import time
import math
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
from scipy import spatial

In [7]:
torch.cuda.set_device(1)
torch.backends.cudnn.benchmark = True
net = Model().cuda()
restore = False
optimizer = torch.optim.SGD(net.parameters(), lr=5e-3, momentum=0.9)
n_epochs = 50
batch_size = 12
num_workers = 12
descriptor_size = 64
bandwith = 100
net_input_size = 2*bandwith
n_features = 2
cache = 50
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
model_save = 'net_params_new_1.pkl'
summary(net, input_size=[(2, 200, 200), (2, 200, 200), (2, 200, 200)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     S2Convolution-1    [-1, 10, 100, 100, 100]              10
              ReLU-2    [-1, 10, 100, 100, 100]               0
       BatchNorm3d-3    [-1, 10, 100, 100, 100]              20
    SO3Convolution-4       [-1, 20, 50, 50, 50]              20
              ReLU-5       [-1, 20, 50, 50, 50]               0
       BatchNorm3d-6       [-1, 20, 50, 50, 50]              40
    SO3Convolution-7       [-1, 60, 40, 40, 40]              60
              ReLU-8       [-1, 60, 40, 40, 40]               0
       BatchNorm3d-9       [-1, 60, 40, 40, 40]             120
   SO3Convolution-10      [-1, 100, 20, 20, 20]             100
      BatchNorm3d-11      [-1, 100, 20, 20, 20]             200
             ReLU-12      [-1, 100, 20, 20, 20]               0
   SO3Convolution-13      [-1, 200, 10, 10, 10]             200
      BatchNorm3d-14      [-1, 200, 10,

Initialize the data source

In [ ]:
n_data = 1000
#ds = DataSource("/mnt/data/datasets/Spherical/training", cache)
ds = DataSource('/media/scratch/berlukas/spherical/test', n_data)
ds.load(n_data)
train_set = TrainingSet(ds, restore, bandwidth)
print("Total size: ", len(train_set))
split = DataSplitter(train_set, restore, shuffle=True)

train_loader, val_loader, test_loader = split.get_split(batch_size=batch_size, num_workers=num_workers)
train_size = split.get_train_size()
val_size = split.get_val_size()
test_size = split.get_test_size()
print("Training size: ", train_size)
print("Validation size: ", val_size)
print("Testing size: ", test_size)